# Dust Detection from Remote Sensing Images
In this notebook input images are used from MODIS Terra satellite from NASA. Here one sample .hdf file is used for input.
There is one data folder where the .hdf file is located. That file is downloaded from NASA LAADS DAAC.

##### All the Necessary packages and libraries are loaded

In [ ]:
from scipy.signal import cspline2d
import imageio as img
from PIL import Image
import cv2

%matplotlib notebook

from satpy import Scene, MultiScene, available_readers, available_writers, find_files_and_readers
from satpy.readers import modis_l1b, modis_l2
from satpy.writers import get_enhanced_image
from satpy.composites import GenericCompositor

from dask.diagnostics import ProgressBar
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyresample import geometry

import os
import warnings

import matplotlib.pyplot as plt
import earthpy as et
import earthpy.plot as ep

from datetime import datetime

from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from sklearn.model_selection import train_test_split

from keras.models import Model, load_model
from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.preprocessing.image import img_to_array, load_img

import tensorflow as tf
# import statements
import torch
import torch.nn as nn
from torch.utils import data
from torchvision import datasets, transforms, models
import numpy as np
import argparse
from tqdm import tqdm
import time

from tensorboardX import SummaryWriter

# from utils import load_checkpoint, save_checkpoint, ensure_dir
# from model import MyModel

## Module 1: Remote Sensing MODIS Data
##### Checking the directory to find the location of data

In [ ]:
#####################
# LOADING DATASET
#####################

def content_from_path(path):
    return os.listdir(path)

def generate_data_from_path(path):
    images = []
    dataset_path = content_from_path(path)
    
    for item in dataset_path:
        images.append(("Yes", item))
        
    return images
     
def generate_data_from_folder():
    images = []
    
    for item in dataset_path:
        path = 'nasa' + '/' + item
        if os.path.isdir(path):  
            image = os.listdir(path)
            images.append(("Yes", str(path + '/' + image[0])))
    return images

In [ ]:
#####################
# MAKING DATAFRAME
#####################

images = generate_data_from_path('nasa-calibrated-resize');
dust_df = pd.DataFrame(data=images, columns=["Dust", "image"])
dust_df

In [ ]:
##################################################
# MAKING HORIZONTAL SPLITTED DATASET
##################################################

counter = 0
for item in dust_df['image']:
    
    input_path = "nasa-calibrated-resize/" + item
    if input_path.endswith('DS_Store') or os.path.isdir(input_path):
        continue
    outputPath = "horizontal/"
    
    img = cv2.imread(input_path)
    im = Image.open(input_path)
    x_width, y_height = im.size
    # print(x_width, y_height)
    
    outputFileFormat = "{0}-{1}.jpg"
    baseName = "cropped"
    
    split = 10
    edges = np.linspace(0, x_width, split + 1)
    # print(edges)
    # print(edges[:-1])
    # print(edges[1:])
    for start, end in zip(edges[:-1], edges[1:]):
        box = (start, 0, end, y_height)
        a = im.crop(box)
        x_width, y_height = a.size
        # print(x_width, y_height)
        a.load()
        outputName = os.path.join(outputPath, outputFileFormat.format(baseName, counter + 1))
        counter = counter + 1
        # print(outputName)
        a.save(outputName, "JPEG")
        
horizontal_images = generate_data_from_path('horizontal');
dust_df_horizontal = pd.DataFrame(data=horizontal_images, columns=["Dust", "image"])
dust_df_horizontal

In [ ]:
for item in dust_df_horizontal['image']:
    
    input_path = "horizontal/" + item
    if input_path.endswith('DS_Store') or os.path.isdir(input_path):
        continue
    
    img = cv2.imread(input_path)
    im = Image.open(input_path)
    # im.show()
    x_width, y_height = im.size
    print(x_width, y_height)
    # print(img.shape)
    # print(input_path)
    if x_width != 128:
        print(input_path)
        os.remove(input_path)
    # break

In [ ]:
##################################################
# MAKING VERTICAL SPLITTED DATASET
##################################################

counter = 0
for item in dust_df_horizontal['image']:
    
    input_path = "horizontal/" + item
    if input_path.endswith('DS_Store') or os.path.isdir(input_path):
        continue
    outputPath = "horizontal/final"
    
    img = cv2.imread(input_path)
    im = Image.open(input_path)
    x_width, y_height = im.size
    # print(x_width, y_height)
    
    outputFileFormat = "{0}-{1}.jpg"
    baseName = "cropped"
    
    split = 15
    edges = np.linspace(0, y_height, split + 1)
    # print(edges)
    # print(edges[:-1])
    # print(edges[1:])
    for start, end in zip(edges[:-1], edges[1:]):
        box = (0, start, x_width, end)
        a = im.crop(box)
        a.load()
        outputName = os.path.join(outputPath, outputFileFormat.format(baseName, counter + 1))
        counter = counter + 1
        # print(outputName)
        a.save(outputName, "JPEG")
        
final_images = generate_data_from_path('horizontal/final');
dust_df_final = pd.DataFrame(data=final_images, columns=["Dust", "image"])
dust_df_final

In [ ]:
for item in dust_df_final['image']:
    
    input_path = "horizontal/final/" + item
    if input_path.endswith('DS_Store') or os.path.isdir(input_path):
        continue
    count = 1
    img = cv2.imread(input_path)
    im = Image.open(input_path)
    # im.show()
    x_width, y_height = im.size
    print(x_width, y_height)
    # print(item.index)
    if y_height != 128:
        print(img.shape)
        os.remove(input_path)
    # break

In [ ]:
##########################################
# MAKING FINAL DATAFRAME
##########################################

images = generate_data_from_path('horizontal/final/');
dust_df_final_calibrated = pd.DataFrame(data=images, columns=["Dust", "image"])
dust_df_final_calibrated

In [ ]:
for item in dust_df_final_calibrated['image']:
    
    input_path = "horizontal/final/" + item
    if input_path.endswith('DS_Store') or os.path.isdir(input_path):
        continue
    count = 1
    img = cv2.imread(input_path)
    im = Image.open(input_path)
    # im.show()
    x_width, y_height = im.size
    # print(x_width, y_height)

In [ ]:

path = 'nasa/'

im_size = 60
resize_images = []
labels = ["dust"]

for i in dataset_path:
    data_path = path + str(i)
    
    # print(data_path)
    if os.path.isdir(data_path):
        filenames = [i for i in os.listdir(data_path)]
        # print(filenames)
        
        for f in filenames:
        
            img = cv2.imread(data_path + "/" + f)

            print(img)

            img = cv2.resize(img, (im_size, im_size))

            print(i)


            resize_images.append(img)
                
resize_images


In [ ]:
images = np.array(resize_images)
images.shape

In [ ]:
images = images.astype('float32') / 255.0
plt.

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
y = dust_df["Dust"].values

y_labelencoder = LabelEncoder()
y = y_labelencoder.fit_transform(y)
print(y)

In [ ]:
y = y.reshape(-1, 1)
onehotencoder = OneHotEncoder()
Y = onehotencoder.fit_transform(y)
print(Y)

In [ ]:
os.getcwd()

##### Changing the folder to data to get access of the .hdf file
This file is downloaded before from NASA LAADS DAAC. Changing the directory should be executed only once otherwise it will show error because it has already changed the directory.


In [ ]:
# Set working directory to data to get access of the file
# os.chdir(os.path.join(os.getcwd(), 'data'))

# Accessing file for processing
filename = os.path.join("MOD021KM.A2021092.0020.006.2021092134055.hdf")
filenames = [filename]

##### Loading data to scene object from SatPy. 
SatPy is used here for processing remote sensing images
Printing available dataset names which is required for listing the bands
Here total 36 bands information are available

In [ ]:
# MODIS scene object using the file retrieved from data folder
modis_scene = Scene(reader='modis_l1b', filenames=filenames)
modis_scene.available_dataset_names()
# modis_scene.unload

##### Method: Band Details and Plotting
This is the custom method which takes band no and color map as parameter and shows all the necessary information about that particular band. The second method plot_band() takes the band no as parameter and plot the band.

In [ ]:
def band_details(band_no, cmap):
    
    print("Band no: ", band_no)
    print("Platform name: ", modis_scene[band_no].attrs['platform_name'])
    print("Dimension: ", modis_scene[band_no].dims)
    print("No of dimension: ", modis_scene[band_no].ndim)
    print("Wavelength: ", modis_scene[band_no].wavelength)
    print("Calibration: ", modis_scene[band_no].calibration)
    print("Maximum value: ", modis_scene[band_no].max().values)
    
    modis_scene[band_no].plot.imshow(cmap=cmap)
    plt.title("Band-{}".format(band_no))

def plot_band(band_no):

    plt.figure()
    modis_scene.load([band_no])
    band_details(band_no, 'cividis')
    modis_scene[band_no]

In [ ]:
band_no = '3'
plot_band(band_no)

### Data Extraction
Image data is extrcted from the metadata of MODIS hdf file

In [ ]:
data = modis_scene[band_no]
data

### Resizing Image
The dimension of input image is too high so the reduced dimension of image is used by resizing it to (128, 128) where original image dimension was (2030, 1354). Finally image value is normalized by dividing by 255. Image is first converted to matrix and then again to numpy array to match the dimension.

In [ ]:
iteration = 100

image = resize(img_to_array(modis_scene[band_no]), (128, 128),  mode = 'constant', preserve_range = True)
image = image/255.0
image.shape
image = np.matrix(image)
image = np.array(image)
image


In [ ]:
# Plotting the image after resizing
plt.figure()
plt.imshow(image, cmap = 'viridis')
plt.title('Band - ' + band_no)

In [ ]:
modis_scene.unload